In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import importlib

import commons
import utils
from models import SynthesizerTrn

from scipy.io.wavfile import write

def get_text(text, hps, text_to_sequence_func):
    text_norm = text_to_sequence_func(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

ImportError: Numba needs NumPy 1.22 or greater. Got NumPy 1.21.

In [3]:
# Load config
hps = utils.get_hparams_from_file("./logs/uudb_6/config.json")

# Load text module and symbols
text_module = importlib.import_module(hps.data.text_module)
symbols = text_module.symbols
text_to_sequence = text_module.text_to_sequence

# Build model
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

# Load checkpoint
# IMPORTANT: Change the path to your trained UUDB model checkpoint
_ = utils.load_checkpoint("./logs/uudb_6/G_700000.pth", net_g, None)

Mutli-stream iSTFT VITS


/home/synology/naru/work/ms2/MB-iSTFT-VITS/utils.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')


In [6]:
# Load config
hps = utils.get_hparams_from_file("./logs/uudb_6/config.json")

# Load text module and symbols
text_module = importlib.import_module(hps.data.text_module)
symbols = text_module.symbols
text_to_sequence = text_module.text_to_sequence

# Build model
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

# Load checkpoint
# IMPORTANT: Change the path to your trained UUDB model checkpoint
_ = utils.load_checkpoint("./logs/uudb_6/G_700000.pth", net_g, None)

Mutli-stream iSTFT VITS


In [10]:
import time
# Synthesize Japanese text
text_to_synthesize = "[エト]アノメノマエニドウグガアルトオモウノダケド"
start_time = time.time()
# Phonemize
# The text_module is loaded in the previous cell
cleaner_names = hps.data.text_cleaners
cleaner = getattr(text_module.cleaners, cleaner_names[0])
phonemized_text = cleaner(text_to_synthesize)

print(f"Original text: {text_to_synthesize}")
print(f"Phonemized: {phonemized_text}")

stn_tst = get_text(text_to_synthesize, hps, text_to_sequence)

with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

Original text: [エト]アノメノマエニドウグガアルトオモウノダケド
Phonemized: [ e t o  ]a n o m e n o m a e n i d o u g u g a a r u t o o m o u n o d a k e d o
Elapsed time: 0.03 seconds
